# Estimate Total Costings

https://openai.com/pricing

####  GPT4
| Model | Input | Output |
| -- | -- | -- |
| 8K context | $0.03 / 1K tokens | $0.06 / 1K tokens |
| 32K context | $0.06 / 1K tokens | $0.12 / 1K tokens|

#### GPT3.5-turbo
| Model | Input | Output |
| -- | -- | -- |
|4K context	| $0.0015 / 1K tokens |$0.002 / 1K tokens |
|16K context | $0.003 / 1K tokens | $0.004 / 1K tokens |

In [56]:
import os
import boto3
import json
from dotenv import load_dotenv


In [2]:
load_dotenv()

aws_access_key = os.environ['aws_access_key']
aws_secret_key = os.environ['aws_secret_key']
aws_region = os.environ['aws_region']
currents_api_key = os.environ['currents_api_key']

In [13]:
s3 = boto3.client(
        's3',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        region_name=aws_region)

bucket_name = 'glimpse-landing-dev'
objs = s3.list_objects_v2(Bucket=bucket_name)['Contents']

In [26]:
all_feeds = []

for obj in objs:
    response = s3.get_object(Bucket=bucket_name, Key=obj['Key'])
    file_content = response['Body'].read().decode('utf-8')
    json_content = json.loads(file_content)
    all_feeds.append(json_content)

In [46]:
all_news = [r['news'] for r in all_feeds]
all_titles = [[news['title'] for news in day] for day in all_news]
all_words = [' '.join(titles) for titles in all_titles]

In [51]:
est_tokens = [len(article.split(' ')) for article in all_words]

[330, 380, 404]